In [51]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from IPython.display import clear_output
from IPython.display import display

import pandas as pd
import os

import plotly.graph_objs as go
import plotly.express as px


import warnings
warnings.filterwarnings('ignore')

In [46]:
fn = "https://www.eia.gov/electricity/data/eia860m/xls/october_generator2019.xlsx"
desired_columns = ['Entity ID', 'Entity Name', 'Plant ID', 'Plant Name', 'Sector',
       'Plant State', 'Generator ID', 'Unit Code', 'Nameplate Capacity (MW)',
       'Net Summer Capacity (MW)', 'Net Winter Capacity (MW)', 'Technology',
       'Energy Source Code', 'Prime Mover Code', 'Operating Month',
       'Operating Year', 'Status', 'County', 'Latitude', 'Longitude',
        'Balancing Authority Code']

In [47]:
ogDF = pd.read_excel(fn, skipfooter=1, header=1, usecols=desired_columns)

In [48]:
ogDF.columns

Index(['Entity ID', 'Entity Name', 'Plant ID', 'Plant Name', 'Sector',
       'Plant State', 'Generator ID', 'Unit Code', 'Nameplate Capacity (MW)',
       'Net Summer Capacity (MW)', 'Net Winter Capacity (MW)', 'Technology',
       'Energy Source Code', 'Prime Mover Code', 'Operating Month',
       'Operating Year', 'Status', 'County', 'Latitude', 'Longitude',
       'Balancing Authority Code'],
      dtype='object')

In [50]:
colorSelector = widgets.Select(options=ogDF.columns, description="Color Code: ")
sizeSelector = widgets.Select(options=ogDF._get_numeric_data().columns, description="Size Code: ")
baas = list(ogDF["Balancing Authority Code"].unique().astype(str))
baaSelector = widgets.SelectMultiple(options=sorted(list(baas))[:-1], description="BAA Filter: ")


plot_button = widgets.Button(description="Plot that data!")

output = widgets.Output()
output2 = widgets.Output()

clickCounter = 0

@plot_button.on_click
def plot_button_clicked(a):
    
    
    with output:
        clear_output()
        prep = ogDF[ogDF["Balancing Authority Code"].isin(list(baaSelector.value))]
        prep["Tooltip"] = colorSelector.value + ": " + prep[colorSelector.value]
                
        f = go.FigureWidget(px.scatter_geo(prep, text="Entity Name", hover_name="Plant Name",locationmode="USA-states",lat="Latitude", lon="Longitude", color=colorSelector.value, size=sizeSelector.value))
        f.update_layout(
            showlegend=True,
            geo = dict(
                scope = 'usa',
                landcolor = "rgb(217, 217, 217)"))
        display(f)
widgets.VBox([baaSelector, widgets.HBox([colorSelector, sizeSelector, plot_button]), output])